In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import mysql.connector as sqcon

### __Defining Functions__

In [3]:
# Function for creating sql connection
def create_sql_conn():
    try:
        conn = sqcon.connect(user = 'student',
              password = 'student',
              host = 'cpane.insaid.co',
            database ='Capstone1')
    except sqcon.Error as e:
        print('Error Creating Database Connection ', e)
    return conn

# Defining the function to ger the table data, it uses create SQL connection fuction
def getTableData(table_name):
    df = pd.DataFrame()
    try:
        conn = create_sql_conn()
        db_cursor = conn.cursor()
        db.cursor.execute('SELECT * FROM' + table_name)
        table_rows = db_cursor.fetchall()
        df = pd.DataFrame(table_rows)
    except sqon.Error as e:
        print("Error Reading data from DB table", e)
    finally:
        db_cursor.close()
        conn.close()
    return df

# Defining the function to find missing data and createa dataframe output
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending=False)
    return pd.concat([total,percent], axis=1, keys=['Total','Percent'])

# Definng the fucntion to plot data on map using lat and long
def plot_on_map(data_to_map):
    center =[data_to_map.iloc[0]['latitude'], data_to_map.iloc[0]['longitude']]
    map_India = folium.Map(location = center, zoom_start=4)
    for index, phone_loc in data_to_map.iterrows():
        location = [phone_loc['latitude'], phone_loc['longitude']]
        folium.Marker(location, popup = f'City:{phone_loc["city"]}').add_to(map_India)
        return map_India
    

### Assignment 1:
#### Reading and Understanding Data
- __Tasks__
- Load the datatables form csv file and sql table into the note book
- Read the data available
- Find the issues with data

### Reading data from MS SQL database

In [7]:
# Reading user data contained in 'gender_age_train' data from database
# Name the columns
# Name databframe as 'user'

user = getTableData('gender_age_train')
user.columns = ['device_id', 'gender', 'age', 'age_group']

In [9]:
# Reading device data contained in 'phone_brand_device_model'
# Name the columns
# Name the dataframe 'device'

device = getTableData('phone_brand_device_model')
device.columns = ['device_id','brand', 'model']

### Reading from local file system
- Data given by INSAID in csv format
- File name : events_data.csv
- Firle location: File is stored in the same location as this notebook

In [11]:
# Reading events data from local storage
# Device ID is in exponential form and is converted to str
# Name Dataframe as 'event'

event =pd.read_csv('events_data.csv', dtype={'device_id':np.str})

## __Basic Analysis of all Data__

### User Data

In [12]:
user.head(5)

,device_id,gender,age,age_group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [14]:
user.shape

(74645, 4)

In [44]:
user.nunique()

device_id    74645
gender           2
age             85
age_group       12
dtype: int64

In [34]:
missing_data(user)

,Total,Percent
age_group,0,0.0
age,0,0.0
gender,0,0.0
device_id,0,0.0


In [19]:
user['age_group'].value_counts(sort=True)

M23-26    9605
M32-38    9476
M39+      8581
M22-      7488
M29-31    7309
F33-42    5561
M27-28    5445
F23-      5050
F29-32    4628
F43+      4194
F24-26    4190
F27-28    3118
Name: age_group, dtype: int64

In [20]:
user['age'].unique()

array([35, 30, 24, 36, 38, 33, 31, 37, 28, 32, 48, 75, 39, 25, 27, 29, 34,
       22, 26, 47, 44, 21, 42, 64, 46, 23, 20, 41, 43, 51, 49, 53, 59, 45,
       50, 70, 67, 19, 52, 40, 63, 60, 61, 78, 56, 65, 58, 17, 16, 18, 71,
       54, 76, 62, 73, 66, 55, 57, 83, 84, 81, 80, 79, 77, 74, 13, 68, 89,
       87, 69, 88, 72, 82, 90, 96, 15,  1, 12, 14, 10, 85, 86, 11,  6, 94])

In [23]:
user[user['age'] < 10].count()

device_id    5
gender       5
age          5
age_group    5
dtype: int64

In [24]:
user[user['age'] < 10]

,device_id,gender,age,age_group
10385,3553057874282315257,M,1,M22-
60779,-8513863533705424532,M,6,M22-
60831,-5323874218045578676,M,6,M22-
60832,3646841514566479686,M,6,M22-
60833,5222152119137955379,M,6,M22-


#### __Observations on user data__
- There are __74645__ rows and __4__ columns in data.
- '# of unique devices are __74645__
- There are no missing data or null values
- Data types are in order

__Age_Group__
- Age group contains letters and numerals which are unnecesary considering gender is givern
- Grouping seems arbitary and of irregualr intervals
__Age__
- Age varies from 01 to 96
- There are __5__ users, one with age __01__ and four with __6__ as age.
- This may need further analysis

__Actions__
- Age group may be deleted and regrouped as per analysis planned
- Ages less than 10 may be considered errous entry and replaced with mode or even left as such to detect anomaly in use/sales.


### Device Data

In [25]:
device.head()

,device_id,brand,model
0,1877775838486905855,vivo,Y13
1,-3766087376657242966,小米,V183
2,-6238937574958215831,OPPO,R7s
3,8973197758510677470,三星,A368t
4,-2015528097870762664,小米,红米Note2


In [27]:
device.shape

(87726, 3)

In [43]:
device.nunique()

device_id    87726
brand          116
model         1467
dtype: int64

In [35]:
missing_data(device)

,Total,Percent
model,0,0.0
brand,0,0.0
device_id,0,0.0


In [32]:
device['brand'].value_counts()

小米      21645
三星      19211
华为      15181
vivo     7596
OPPO     6717
        ...  
MIL         1
瑞高          1
帷幄          1
凯利通         1
大显          1
Name: brand, Length: 116, dtype: int64

#### __Observations on device data__
- There are __87726__ rows and __3__ columns in data.
- '# of unique devices are __87726__
- There are no missing data or null values
- Data types are in order

__Brand__

- There are __1467__ brands 
- Most brand names are in Chineese language

__Model__

- Model seems insignificant at this stage

__Actions__

- Convert brand names in Chineese to English language
- Drop column 'model'



### Event Data

In [36]:
event.head(5)

,event_id,device_id,timestamp,longitude,latitude,city,state
0,2765368,2973347786994914304,2016-05-07 22:52:05,77.225676,28.730140,Delhi,Delhi
1,2955066,4734221357723754978,2016-05-01 20:44:16,88.388361,22.660325,Calcutta,WestBengal
2,605968,-3264499652692492784,2016-05-02 14:23:04,77.256809,28.757906,Delhi,Delhi
3,448114,5731369272434022440,2016-05-03 13:21:16,80.343613,13.153332,Chennai,TamilNadu
4,665740,338888002570799963,2016-05-06 03:51:05,85.997745,23.842609,Bokaro,Jharkhand


In [37]:
event.shape

(3252950, 7)

In [42]:
event.nunique()

event_id     3252950
device_id      60865
timestamp     588126
longitude      60095
latitude       60185
city             933
state             32
dtype: int64

In [41]:
missing_data(event)

,Total,Percent
device_id,453,0.013926
latitude,423,0.013004
longitude,423,0.013004
state,377,0.011589
city,0,0.000000
timestamp,0,0.000000
event_id,0,0.000000


#### Analysis of missing data based on city
- City has no missing values, its prudent to analyse missing data with respect to city

In [48]:
event[event['latitude'].isnull()]['city'].value_counts()

Jaipur           66
Chennai          63
Pune             63
Indore           63
Visakhapatnam    63
Delhi            63
Araria           14
Bagaha           14
Moga             14
Name: city, dtype: int64

In [49]:
event[event['longitude'].isnull()]['city'].value_counts()

Jaipur           66
Chennai          63
Pune             63
Indore           63
Visakhapatnam    63
Delhi            63
Araria           14
Bagaha           14
Moga             14
Name: city, dtype: int64

In [50]:
event[event['state'].isnull()]['city'].value_counts()

Delhi            64
Pune             63
Indore           51
Jaipur           49
Chennai          47
Visakhapatnam    47
Gangarampur      26
Arambagh         16
Channapatna      14
Name: city, dtype: int64

#### Observation on event data
- Contains __3252950__ rows and __7__ columns
- Data gives the time and location based on city, state and lat/long refrence of the event
- There are __60865__ devices that has created these events
- Events are recorded from __933__ cities across __32__ states of India

####  __That is there are _3252950_ events created using _87726_ devices by _74645_ users__

__Missing Data__
- Device_Id : __453__
- Lat/long  : __423__
- State     : __377__

__Lat/Long__
- Both lat and long are missing from same rows
- 09 cities account for all missing lat/long

__State__
- 09 cities account for of all missing states
- They are not same as missing lat/long

__Actions__
- __Device_id__ is the common point in all three data sets
- Missing data in both states and lat/long can be filled using city
- Mode of the lat/long for the cities where data is missing can be used fo the same.
- The tables may be mereged to form a single dataframe for further analysis based on device data.


### Finding common unique id in all three tables
__This can be done using two ways__

   - By creating merged datasets
   - By code finding intersection of values


#### Method 1: By merging datasets

In [53]:
# Merging device and user table to include all rows
device_user_outer = pd.merge(device, user, how ='outer')

In [150]:
device_user_outer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87754 entries, 0 to 87753
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   device_id  87754 non-null  int64  
 1   brand      87726 non-null  object 
 2   model      87726 non-null  object 
 3   gender     74645 non-null  object 
 4   age        74645 non-null  float64
 5   age_group  74645 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.7+ MB


In [151]:
# Merging device and user data on device id of all devices
device_user = pd.merge(device, user, how ='left', on = 'device_id')

In [152]:
device_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87726 entries, 0 to 87725
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   device_id  87726 non-null  int64  
 1   brand      87726 non-null  object 
 2   gender     74617 non-null  object 
 3   age        74617 non-null  float64
 4   age_group  74617 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 4.0+ MB


#### Method 2 : By code using intersection of values
- Create sets of device id
    - user_set
    - device_set
    - event_set
- Find the intersection of these sets to get common values

In [65]:
# Creating the sets

user_set   = set(user['device_id'].astype(str).unique())
device_set = set(device['device_id'].astype(str).unique())
event_set  = set(event['device_id'].astype(str).unique())

# finding the intersection of above sets using code

user_device_set = user_set.intersection(device_set)
common_device_set = event_set.intersection(user_device_set)


In [67]:
print("Gender Age count : {0}\nPhone Brand Count : {1}\nEvents Device Count : {2}".format(len(user_set), len(device_set), len(event_set)))
print("Common devices between Gender Age and Phone Brand Datasets is : ", len(user_device_set))
print("Common devices between all 3 Datasets is : ", len(common_device_set))

Gender Age count : 74645
Phone Brand Count : 87726
Events Device Count : 60866
Common devices between Gender Age and Phone Brand Datasets is :  74617
Common devices between all 3 Datasets is :  60865


### Observation on Common Device ID
__Common Device ID__
- Since device id is common in all three tables, find the common 
device ids present
 - Between __user__ and __device__ : 74617
 - Between all datasets.           : 60866

## Assignment for Week 2

### Tasks

#### Preprocessing of Data
- Rectify the issues as observed in week one
- Create a single data set
- Analyse and update further observations

### Convert model names in Chineese into English

__Method Followed__
- Copy the brand names to google translator and find English names.
- Create a dataframe with both Chineese and English Names.
- Merge this dataframe to the exisiting device dataframe using brand name.
- Drop coloum with Chineese name and rename the brand name column head to 'brand'

In [69]:
device['brand'].unique()

array(['vivo', '小米', 'OPPO', '三星', '酷派', '联想 ', '华为', '奇酷', '魅族', '斐讯',
       '中国移动', 'HTC', '天语', '至尊宝', 'LG', '欧博信', '优米', 'ZUK', '努比亚', '惠普',
       '尼比鲁', '美图', '乡米', '摩托罗拉', '梦米', '锤子', '富可视', '乐视', '海信', '百立丰',
       '一加', '语信', '海尔', '酷比', '纽曼', '波导', '朵唯', '聆韵', 'TCL', '酷珀', '爱派尔',
       'LOGO', '青葱', '果米', '华硕', '昂达', '艾优尼', '康佳', '优购', '邦华', '赛博宇华',
       '黑米', 'Lovme', '先锋', 'E派', '神舟', '诺基亚', '普耐尔', '糖葫芦', '亿通', '欧新',
       '米奇', '酷比魔方', '蓝魔', '小杨树', '贝尔丰', '糯米', '米歌', 'E人E本', '西米', '大Q',
       '台电', '飞利浦', '唯米', '大显', '长虹', '维图', '青橙', '本为', '虾米', '夏新', '帷幄',
       '百加', 'SUGAR', '欧奇', '世纪星', '智镁', '欧比', '基伍', '飞秒', '德赛', '易派',
       '谷歌', '金星数码', '广信', '诺亚信', 'MIL', '白米', '大可乐', '宝捷讯', '优语', '首云',
       '瑞米', '瑞高', '沃普丰', '摩乐', '鲜米', '凯利通', '唯比', '欧沃', '丰米', '恒宇丰',
       '奥克斯', '西门子', '欧乐迪', 'PPTV'], dtype=object)

In [71]:
# Since google translation API was giving issues with large datasets,
# The unique brand names were translated extrenally and then copied into 
# notebook to create a dictionary.

brand_list={'brand':['vivo', '小米', 'OPPO', '三星', '酷派', '联想 ', '华为', '奇酷', '魅族', '斐讯',
       '中国移动', 'HTC', '天语', '至尊宝', 'LG', '欧博信', '优米', 'ZUK', '努比亚', '惠普',
       '尼比鲁', '美图', '乡米', '摩托罗拉', '梦米', '锤子', '富可视', '乐视', '海信', '百立丰',
       '一加', '语信', '海尔', '酷比', '纽曼', '波导', '朵唯', '聆韵', 'TCL', '酷珀', '爱派尔',
       'LOGO', '青葱', '果米', '华硕', '昂达', '艾优尼', '康佳', '优购', '邦华', '赛博宇华',
       '黑米', 'Lovme', '先锋', 'E派', '神舟', '诺基亚', '普耐尔', '糖葫芦', '亿通', '欧新',
       '米奇', '酷比魔方', '蓝魔', '小杨树', '贝尔丰', '糯米', '米歌', 'E人E本', '西米', '大Q',
       '台电', '飞利浦', '唯米', '大显', '长虹', '维图', '青橙', '本为', '虾米', '夏新', '帷幄',
       '百加', 'SUGAR', '欧奇', '世纪星', '智镁', '欧比', '基伍', '飞秒', '德赛', '易派',
       '谷歌', '金星数码', '广信', '诺亚信', 'MIL', '白米', '大可乐', '宝捷讯', '优语', '首云',
       '瑞米', '瑞高', '沃普丰', '摩乐', '鲜米', '凯利通', '唯比', '欧沃', '丰米', '恒宇丰',
       '奥克斯', '西门子', '欧乐迪', 'PPTV'], 
    'brand_eng':['vivo','Xiaomi','OPPO','Samsung','Coolpad','Lenovo','Huawei','Qiku','Meizu','Phicom',
       'China Mobile','HTC','Tianyu','Extreme Bao','LG','Oberxin','Yumi','ZUK','Nubia','HP',
       'Nibiru','Meitu','Xiangmi','Motorola','Mengmi','Hammer','InFocus','LeTV','Hisense','Bai Li Feng',
       'OnePlus','Yuxin','Haier','Cubbie','Newman','Bird','Dove','Lingyun','TCL','Copper','Aipel ',
       'LOGO','green onion','fruit rice','ASUS','Onda','Aiyouni','Konka','Yougo','Banghua','Cyber ​​Yuhua',
       'Black Rice','Lovme','Pioneer','E School','Shenzhou','Nokia','Pnair','Candy Gourd','Yitong','Ouxin',
       'Mickey','Cubbie Rubik','Blue Devils','Little Poplar','Bairfeng','Nuomi','Mi Song','E People E Book','Simi','Big Q' ,
       'Taipower','Philips','Weimi','Daxian','Changhong','Vitu','Green Orange','Originally','Xia Mi','Amoi','Wei Mi',
       'Bajia','SUGAR','Ouqi','Century Star','Zhimei','Obi','Kivu','Femtosecond','Desai','Yipai',
       'Google','Venus Digital','Guangxin','Noah Xin','MIL','White Rice','Coke','Baojiexun','Youyu','Shouyun',
       'Rimei','Rigao','Wopfung','Mole','Fresh Rice','Kailyton','Weibi','Ouwo','Fengmi','Hengyufeng ',
       'Ox','Siemens','Orlandi','PPTV']}
brand_df=pd.DataFrame.from_dict(brand_list)
brand_df

,brand,brand_eng
0,vivo,vivo
1,小米,Xiaomi
2,OPPO,OPPO
3,三星,Samsung
4,酷派,Coolpad
...,...,...
111,恒宇丰,Hengyufeng
112,奥克斯,Ox
113,西门子,Siemens
114,欧乐迪,Orlandi


In [72]:
device = pd.merge(device,brand_df,how='left',on='brand')
device.head(3)

,device_id,brand,model,brand_eng
0,1877775838486905855,vivo,Y13,vivo
1,-3766087376657242966,小米,V183,Xiaomi
2,-6238937574958215831,OPPO,R7s,OPPO


In [76]:
# For the time being, only brand is being changed to English and model column is dropped
# However, the model colum may be changed in a similar manner
device.drop('brand',axis=1,inplace=True)

In [80]:
device.drop('model',axis=1,inplace=True)

In [77]:
device.rename(columns={'brand_eng':'brand'}, inplace = True)

In [81]:
device.head()

,device_id,brand
0,1877775838486905855,vivo
1,-3766087376657242966,Xiaomi
2,-6238937574958215831,OPPO
3,8973197758510677470,Samsung
4,-2015528097870762664,Xiaomi


### Replacing Missing Values in event dataset
- __City__ is considered as the anchor for filling missing values in lat/long and state

### Replacing Latitude/Longitude

In [84]:
# Create a list of cities where lat/long is missing

missing_location_list = event[event.latitude.isnull()]['city'].unique()

In [86]:
print(missing_location_list)

['Delhi' 'Indore' 'Visakhapatnam' 'Chennai' 'Pune' 'Jaipur' 'Araria'
 'Moga' 'Bagaha']


In [99]:
# Create a dictionary of city and modes of lat and long

location_dict = {}

for city in missing_location_list:
    mode_latitude  = (event.loc[event['city']== city]['latitude']).mode()[0]
    mode_longitude = (event.loc[event['city']== city]['longitude']).mode()[0]
    location_dict[city]  = [mode_latitude, mode_longitude]
print(location_dict)

{'Delhi': [28.749820000000003, 77.29399099999998], 'Indore': [22.809817000000002, 75.89546299999998], 'Visakhapatnam': [17.829748000000006, 83.326714], 'Chennai': [13.128174, 80.359796], 'Pune': [18.616127, 73.926254], 'Jaipur': [26.949746, 75.89405699999998], 'Araria': [26.211919, 87.570736], 'Moga': [30.904178, 75.268748], 'Bagaha': [27.17739700000001, 84.1409]}


In [106]:
# Filling the missing values

for city in location_dict:
    event.loc[((event['city']== city) & (event['latitude'].isnull())), 'latitude'] = float(location_dict[city][0])
    event.loc[((event['city']== city) & (event['longitude'].isnull())),'longitude'] = float(location_dict[city][1])

### Replacing State

In [108]:
# Create a list of missing State

missing_state_list = event[event['state'].isnull()]['city'].unique()

In [110]:
print(missing_state_list)

['Pune' 'Visakhapatnam' 'Indore' 'Chennai' 'Delhi' 'Channapatna' 'Jaipur'
 'Gangarampur' 'Arambagh']


In [113]:
# Create a dictionary of missing state mapped to city

missing_state_dict = {}

for city in missing_state_list:
    missing_state_dict[city] = (event.loc[(event['city'] == city) & (~event['state'].isnull())]['state']).reset_index(drop=True)[0]
    
print(missing_state_dict)

{'Pune': 'Maharashtra', 'Visakhapatnam': 'AndhraPradesh', 'Indore': 'MadhyaPradesh', 'Chennai': 'TamilNadu', 'Delhi': 'Delhi', 'Channapatna': 'Karnataka', 'Jaipur': 'Rajasthan', 'Gangarampur': 'WestBengal', 'Arambagh': 'WestBengal'}


In [116]:
# Updating the missing state list

for city in missing_state_dict:
    event.loc[((event['city'] == city) & (event['state'].isnull())), 'state'] = str(missing_state_dict[city])
    #event.loc[((event['city'] == city) & (event['state'].isnull())), 'state'] = str(city_state_dict[city])

### Replacing the missing device_ids

- Since devices_ids can be traced using lat/long, combining them to a single column will help 
- Therefore, create a temp column containing both lat and long
- Use the mode of location for the missing device_id to fill the missing dvice_id

In [133]:
# Creating new col 'joint_loc'

event['joint_loc']= [''.join([str(x), '-', str(y)]) for x, y in zip(event['latitude'],event['longitude'])]

In [121]:
event.head(3)

,event_id,device_id,timestamp,longitude,latitude,city,state,joint_loc
0,2765368,2973347786994914304,2016-05-07 22:52:05,77.225676,28.730140,Delhi,Delhi,28.73014-77.22567599999998
1,2955066,4734221357723754978,2016-05-01 20:44:16,88.388361,22.660325,Calcutta,WestBengal,22.660325-88.388361
2,605968,-3264499652692492784,2016-05-02 14:23:04,77.256809,28.757906,Delhi,Delhi,28.757906-77.25680899999998


In [122]:
event.nunique()

event_id     3252950
device_id      60865
timestamp     588126
longitude      60095
latitude       60185
city             933
state             32
joint_loc      60869
dtype: int64

In [131]:
# Create Missing device list
missing_device_list = event[event['device_id'].isnull()]['joint_loc'].unique()

In [132]:
print(missing_device_list)

['22.814519-75.88295600000002' '26.948689-75.888487' '26.960796-75.846007'
 '22.777781-75.92333199999999' '31.561746999999997-75.99255099999998'
 '18.628057-73.862756' '18.566925-73.86016500000002' '17.805195-83.357991'
 '28.719966000000003-77.29248100000002' '18.614812-73.92649899999998'
 '13.153332-80.34361299999998' '13.149176-80.30927199999998'
 '28.721053-77.27481399999998' '17.752819-83.371738' '22.817526-75.95805'
 '17.822906-83.342711' '26.95399-75.836167' '13.189053-80.33543499999998'
 '21.194283-73.16934499999998' '21.790693-70.686387'
 '28.728888-77.30315300000002']


In [136]:
# Create a dictionary of lat_long and device id

device_id_loc_dict = {}

for loc in missing_device_list:
    device_id_loc_dict[loc] = (event.loc[event['joint_loc'] == loc]['device_id'].mode()[0])

print(device_id_loc_dict)

{'22.814519-75.88295600000002': '3132861355949762783', '26.948689-75.888487': '-917725110041294385', '26.960796-75.846007': '-8460337302442785825', '22.777781-75.92333199999999': '7597735768595924821', '31.561746999999997-75.99255099999998': '1750778632182066836', '18.628057-73.862756': '9027085705222633039', '18.566925-73.86016500000002': '8539870903150472243', '17.805195-83.357991': '6844192269280790123', '28.719966000000003-77.29248100000002': '5098778421671837341', '18.614812-73.92649899999998': '177158571165078698', '13.153332-80.34361299999998': '5731369272434022440', '13.149176-80.30927199999998': '7662139143719919126', '28.721053-77.27481399999998': '3724654925765159056', '17.752819-83.371738': '-3130903177076173323', '22.817526-75.95805': '-2399250255599832969', '17.822906-83.342711': '6619714762103551896', '26.95399-75.836167': '3562355872640618177', '13.189053-80.33543499999998': '7983871429718484702', '21.194283-73.16934499999998': '-8215770519233685145', '21.790693-70.6863

In [137]:
# Update the missing values in device_id
for loc in device_id_loc_dict:
   
    event.loc[((event['joint_loc'] == loc) & (event['device_id'].isnull())), 'device_id'] = str(device_id_loc_dict[loc])
    
    

In [141]:
event.drop('joint_loc', axis =1, inplace =True)

In [149]:
missing_data(event)

,Total,Percent
state,0,0.0
city,0,0.0
latitude,0,0.0
longitude,0,0.0
timestamp,0,0.0
device_id,0,0.0
event_id,0,0.0


## Creating Combined data set of all three datasets

In [170]:
user['device_id']=user['device_id'].astype(str)

In [173]:
event_user = pd.merge(event, user, how = 'left', on = 'device_id')

In [174]:
event_user.head()

,event_id,device_id,timestamp,longitude,latitude,city,state,gender,age,age_group
0,2765368,2973347786994914304,2016-05-07 22:52:05,77.225676,28.730140,Delhi,Delhi,M,35,M32-38
1,2955066,4734221357723754978,2016-05-01 20:44:16,88.388361,22.660325,Calcutta,WestBengal,M,30,M29-31
2,605968,-3264499652692492784,2016-05-02 14:23:04,77.256809,28.757906,Delhi,Delhi,M,24,M23-26
3,448114,5731369272434022440,2016-05-03 13:21:16,80.343613,13.153332,Chennai,TamilNadu,M,26,M23-26
4,665740,338888002570799963,2016-05-06 03:51:05,85.997745,23.842609,Bokaro,Jharkhand,M,30,M29-31


In [177]:
event_user.info(null_counts= True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3252950 entries, 0 to 3252949
Data columns (total 10 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   event_id   3252950 non-null  int64  
 1   device_id  3252950 non-null  object 
 2   timestamp  3252950 non-null  object 
 3   longitude  3252950 non-null  float64
 4   latitude   3252950 non-null  float64
 5   city       3252950 non-null  object 
 6   state      3252950 non-null  object 
 7   gender     3252950 non-null  object 
 8   age        3252950 non-null  int64  
 9   age_group  3252950 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 273.0+ MB


In [178]:
missing_data(event_user)

,Total,Percent
age_group,0,0.0
age,0,0.0
gender,0,0.0
state,0,0.0
city,0,0.0
latitude,0,0.0
longitude,0,0.0
timestamp,0,0.0
device_id,0,0.0
event_id,0,0.0


In [179]:
event_user.nunique()

event_id     3252950
device_id      60865
timestamp     588126
longitude      60095
latitude       60185
city             933
state             32
gender             2
age               85
age_group         12
dtype: int64

In [181]:
device['device_id']=device['device_id'].astype(str)

In [201]:
data = pd.merge(event_user, device, how = 'left', left_on = 'device_id', right_on='device_id')

In [202]:
data.head()

,event_id,device_id,timestamp,longitude,latitude,city,state,gender,age,age_group,brand
0,2765368,2973347786994914304,2016-05-07 22:52:05,77.225676,28.730140,Delhi,Delhi,M,35,M32-38,Yumi
1,2955066,4734221357723754978,2016-05-01 20:44:16,88.388361,22.660325,Calcutta,WestBengal,M,30,M29-31,vivo
2,605968,-3264499652692492784,2016-05-02 14:23:04,77.256809,28.757906,Delhi,Delhi,M,24,M23-26,Xiaomi
3,448114,5731369272434022440,2016-05-03 13:21:16,80.343613,13.153332,Chennai,TamilNadu,M,26,M23-26,HTC
4,665740,338888002570799963,2016-05-06 03:51:05,85.997745,23.842609,Bokaro,Jharkhand,M,30,M29-31,Xiaomi


In [203]:
data.info(null_counts= True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3252950 entries, 0 to 3252949
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   event_id   3252950 non-null  int64  
 1   device_id  3252950 non-null  object 
 2   timestamp  3252950 non-null  object 
 3   longitude  3252950 non-null  float64
 4   latitude   3252950 non-null  float64
 5   city       3252950 non-null  object 
 6   state      3252950 non-null  object 
 7   gender     3252950 non-null  object 
 8   age        3252950 non-null  int64  
 9   age_group  3252950 non-null  object 
 10  brand      3252950 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 297.8+ MB


In [204]:
missing_data(data)

,Total,Percent
brand,0,0.0
age_group,0,0.0
age,0,0.0
gender,0,0.0
state,0,0.0
city,0,0.0
latitude,0,0.0
longitude,0,0.0
timestamp,0,0.0
device_id,0,0.0


In [189]:
data.nunique()

event_id     3252950
device_id      60865
timestamp     588126
longitude      60095
latitude       60185
city             933
state             32
gender             2
age               85
age_group         12
brand            105
dtype: int64

## Selection of states as per INSAID Directive

In [205]:
state_list = ['AndhraPradesh', 'Pondicherry', 'Mizoram', 'AndamanandNicobarIslands', 'Meghalaya', 'HimachalPradesh']

In [206]:
def state_selection(state_list):
    state_series = data.state.isin(state_list)
    state_data = data[state_series]
    state_data = state_data.reset_index(drop=True)
    return state_data

In [207]:
selected_data = state_selection(state_list)

In [208]:
selected_data.head()

,event_id,device_id,timestamp,longitude,latitude,city,state,gender,age,age_group,brand
0,1078723,-512424213979236423,2016-05-02 02:21:20,83.398244,17.768149,Visakhapatnam,AndhraPradesh,M,31,M29-31,Xiaomi
1,280014,-8879643774076165575,2016-05-05 13:06:01,78.155397,16.390327,Wanparti,AndhraPradesh,M,34,M32-38,Samsung
2,2334601,-601883305299256779,2016-05-05 11:17:48,83.380111,17.828583,Visakhapatnam,AndhraPradesh,M,30,M29-31,Huawei
3,2064864,-2764520535330416860,2016-05-03 23:58:20,83.315014,17.825280,Visakhapatnam,AndhraPradesh,F,27,F27-28,Meizu
4,1341801,4986891477794016217,2016-05-07 15:24:58,83.324339,17.778384,Visakhapatnam,AndhraPradesh,M,43,M39+,Xiaomi


In [209]:
missing_data(selected_data)

,Total,Percent
brand,0,0.0
age_group,0,0.0
age,0,0.0
gender,0,0.0
state,0,0.0
city,0,0.0
latitude,0,0.0
longitude,0,0.0
timestamp,0,0.0
device_id,0,0.0


In [210]:
selected_data.nunique()

event_id     329172
device_id      5223
timestamp    240679
longitude      5203
latitude       5210
city            113
state             6
gender            2
age              72
age_group        12
brand            69
dtype: int64